<a href="https://colab.research.google.com/github/Lozkins/ORP/blob/master/0_network_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разделяй и запускай: делим тестовый стенд между департаментами

<!-- Промышленное математическое программирование как теневой рынок, вроде оно есть и работает, но детали никто не расскрывает.  -->

Промышленное математическое программирование - тема раскрученная в академической среде для стандартизированных случаев, но детали реальных внедрений раскрывают редко и спустя много лет.

В настоящей статье делюсь опытом разработки и внедрения в процессы компании оптимизационного решения на базе математического программирования. Материал расширил исследовательскими элементами и локальным мини benchmark'ом.



# Бизнес задача

На входе имеем крупного производителя сетевого оборудования. У которого одним из этапов контроля качества продукции является тестирование пользовательских, нагрузочных или функциональных сценариев работы продукции на собственных тестовых стендах. Об организации этого процесса компании последует дальнейшее повествование.

Сам тестовый стенд представляет из себя физическую сеть составленную из оборудования, тестеров и серверов; соединенных между собой проводами/кабелем различного вида (витая пара, опто-волокно, коаксильный кабель и [другие](https://translated.turbopages.org/proxy_u/en-ru.ru.5e94bb7c-661d04de-01aaec34-74722d776562/https/en.wikipedia.org/wiki/Networking_cable?__ya_mt_enable_static_translations=1)). В рамках задачи не предполагается физическое изменение стенда, поэтому будем воспринимать его как сеть с фиксированным набором атрибутов и свойств.

Различные подразделения компании работают над улучшением определенных характеристик продукции или производят проверку сценариев ее использования. Каждый такой отдел разрабатывает свои тестовые сценарии и выполняет их на тестовом стенде.

Из чего состоит тест? Тест состоит из скрипта и требований к оборудованию на котором он может быть запущен. Содержание скрипта теста - черный ящик и этого понимания достаточно для решения задачи. В свою очередь, требования к оборудованию распадаются на топологию и специальные свойства оборудования/соединений. На основе этих данных происходит маппинг требований теста и участка тестового стенда для его выполнения.

**Проблема бизнеса:**
- тестовый стенд - дорогое удовольствие. Создавать и поддерживать стенды под разные департаменты компании затратно;
- не все подразделения могут постоянно нагружать собственный стенд, что приводит к простоям оборудования.

---

Резюмирую:
- тестовый стенд это сеть из оборудования и соединений;
- у каждого отдела свой набор тестовых сценариев;
- каждому тесту ставится в соответствие набор подсетей тестового стенда, где может быть выполнен тест.

`Задача: необходимо разделить тестовый стенд на непересекающиеся подсети между подразделениями компании.`

Как было до внедрения изменений:
- Планирование разбиения тестового стенда занимало 2-5 чел/дней;
- Разбиение стенда не более чем на 3 департамента.  


# Накладываем задачу на данные

Подойти к решению задачи на основе имеющихся данных было достаточно проблематично. В первую очередь, не доставало понимания на каких участках тестового стенда можно выполнить тот или иной тестовый кейс. Из описания требований теста к оборудованию было трудоемко определить где его можно выполнить на тестовом стенде. Каждый тест это отдельная задача удовлетворения ограничений на сети.

![Переход от требований к набору допустимых подсетей](https://drive.google.com/uc?export=view&id=1HACJ2RUlmEf_d2yMzPjtIQ_ryOkj2Urj)

Второй фактор, который нам не удалось побороть - ожидаемая длительность выполнения теста. Эта информация помогла бы разбивать тестовый стенд согласно ожидаемой нагрузке. Но раскрытие содержимого скрипта тестового сценария находилось в тени от нас. История или прогнозное время выполнения тестового сценария - не собиралась и не анализировалась.

Теперь о человеческой жадности. Доля покрытия своих тестов каждым отделом формировалась индивидуально без общего согласования и приоритетов. Очевидно, каждый хочет перетянуть одеяло на себя. Тут нам пошли на встречу и в процесс добавили судью, который выставлял приоритеты. Дополнительно к приоритетам добавили учет размера множества тестов отдела: чем больше у отдела тестов, тем больше единиц оборудование ему полагается. Назовем это условие справедливого дележа.

Не всегда удается разделить стенд согласно указанным требованиям к покрытию тестов. Возникает вопрос надеженности модели: выдавать ошибку или выдать решение с нарушениями (мягкие ограничения)? Безусловно, второй вариант наиболее практичный с точки зрения пользователя, тем более у отделов есть маркировка - приоритет.

Размер не имеет значения. Выполнение теста на одной единице оборудования равносильна выполнению теста на 5 единицах оборудования. Это облегчающее обстоятельство уводит нас от многокритериальной задачи. Такое допущение принято для упрощения материала статьи, в боевом решение было по другому.

Наша интерпретация условий и наши возможности:

- **Предложение** предварительно расcчитать все или подмножество вариантов назначения теста на участки тестового стенда. Т.е. с каждым тестовым кейсом связать множество допустимых подсетей тестового стенда.

- **Предположение:** все тесты имеют одинаковое время выполнения. Исходили из того что есть. Кол-во единиц оборудования, которое требует тестовый сценарий - игнорируем. В этом случае, нагрузка на тестовый стенд определяется кол-вом выполняемых тестов.

- **Ограничение:** необходимо разбить тестовый стенд таким образом, чтобы каждое подразделение могло выполнить на своем "куске" стенда определенный процент от своего набора тестов (условие минимального покрытия).





# Мат. модель

Выбор методологии решения задачи. Отталкивались от того, на что похожа наша задача и как эту похожую задачу решают. [Задача о разбиении множества](https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html) (Set partitioning proplem, SPP) - разбиение множества элементов на непересекающиеся подмножества - идеально вписывается в нашу постановку. Задачи типа SPP в основном решают программированием в ограничениях, целочисленным программированием или эвристиками.

Целочисленное программирование или программирование в ограничениях предоставляют общий механизм решения задачи на очерченном поле допустимых решений, в то время как эвристический подход идет по заранее определенному пути.

Наш выбор пал на программирование в ограничениях. Потому что часть работ с нас снимает готовый solver, меньше времени требуется на разработку алгоритмов поиска решения и его тестирование. Это позволяет быстрее доставлять результаты бизнесу. Действовали по принципу 20 на 80.

Перейду непосредственно к постановке задачи в виде задачи целочисленного линейного программирования.

## Индексы и множества

$i \in I$ - множество тестовых сценариев;

$j \in J^i$ - множество индексов подсетей тестового стенда, где может быть выполнен тест $i$;

$k \in K$ - множество оборудования из которого состоит тестовый стенд;

$d \in D$ - множество департаментов компании, между которыми требуется разделить тестовый стенд;

$(i, j) \in \Omega^{kd}$ - подмножество подсетей тестовых сценариев, которые принадлежат отделу $d$ и используют оборудование $k$;

## Постоянные величины

$n_{d}$ - кол-во тестовых сценариев отдела $d$, которые необходимо покрыть;

$w_{d}$ - весовой приоритет выполнения объемного покрытия тестовых сценариев для отдела $d$;

$f_{d}$ - минимальное кол-во единиц оборудования, которое нужно передать отделу $d$;

## Решающие переменные

$x_{kd}$ - бинарная переменная, принимает значение 1, если оборудование $k$ назначено отделу $d$; 0 в противном случае;

$y_{id}$ - бинарная переменная, принимает значение 1, если тест $i$ отдела $d$ может быть выполнен на участке тестового стенда закрепленным за отделом; 0 в противном случае;

$z_{ijd}$ - бинарная переменная, принимает значение 1, если все оборудование подсети $j$ теста $i$ отдела $d$ закреплено за отделом; 0 в противном случае;

$p_{d}$ - целочисленная переменная, объем нарушения уровня покрытия тестов для отдела $d$;

| № | Ограничение &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| Описание |
|---|---|---|
| 1 | $$\sum_{i, j \in \Omega^{kd}}  z_{ijd} \le |\Omega^{kd}| x_{kd}, \quad \forall k \in K, d \in D$$ | Тестовый сценарий отдела можно выполнить на подсети, если все оборудование подсети закреплено за отделом |
| 2 | $$\sum_{d \in D} x_{kd} \le 1, \quad \forall k \in K$$ | Каждая единица оборудования может быть закреплена не более чем за одним отделом |
| 3 | $$\sum_{j \in J} z_{ijd} \ge y_{id}, \quad \forall i \in I, d \in D$$| Тестовый сценарий отдела можно выполнить, если хотя бы одна подсеть использует оборудование полностью закрепленное за этим отделом |
| 4 | $$\sum_{i} y_{id} + p_{d} >= n_d, \quad \forall d \in D $$ | Условие покрытия объема тестов для каждого отдела. Ограничение сформулировано мягким с возможностью нарушить (слаковая переменная $p_{d}$) |
| 5 | $$\sum_{k \in K} x_{kd} \ge f_d, \quad \forall d \in D$$ | Условие справедливого дележа согласно объему тестов |

## Целевая функция

- минимизация взвешенного объема нарушений покрытия тестовых сценариев:

$$\min \sum_{d \in D} w_{d} p_{d}$$


# Python реализация

## Данные

Обременение конфиденциальностью не позволяет 3D погружение в задачу. Поэтому воспользуюсь AI (случайным генератором) для дополнения картины недостающими данными.

|dep\_id|tc\_id|sn\_id|device\_id|
|---|---|---|---|
|1|0|0|d\_3|
|1|0|0|d\_6|
|1|0|0|d\_26|
|1|0|0|d\_15|
|1|0|0|d\_19|
|...|...|...|...|

Входной набор данных состоит из одной таблицы с четырьмя полями:
- dep_id - идентификатор отдела/подразделения;
- tc_id - идентификатор тестового сценария;
- sn_id - идентификатор подсети, где может быть выполнен тестовый сценарий;
- device_id - идентификатор оборудования тестового стенда.

Таблица с данными минималистична, но содержит достаточно информации для принятия решения по разбиению тестового стенда. Ниже приведу сводную информацию по входным данным: общее кол-во тестовых сценариев у департамента, суммарное кол-во подсетей для запуска и кол-во задействованных единиц оборудования тестового стенда в подсетях (тестовый стенд состоит из 45 единиц оборудования).

|dep\_id|tc\_cnt|sn\_cnt|device\_unique|$f_d$|
|---|:---:|:---:|:---:|:---:|
|1|44|6803|45|2|
|2|89|14066|45|5|
|3|147|21202|45|8|
|4|184|25021|45|10|
|5|241|33406|45|13|

В модели фигурирует параметр кол-ва тестовых сценариев, которые требуется покрыть у каждого отдела $n_d$. Положим это значение равным 60% от общего кол-ва тестов отдела.

Разные отделы имеют разный объем тестовых сценариев для запуска. Как следствие, нагрузка на подсети отличается. Распределение в зависимости от нагрузки обеспечивает ограничение (5), а параметр $f_d$ указывает минимальное кол-во оборудования для отдела. Значение параметров $f_d$ тоже потребуется сгенерировать.

![Пример тестового стенда](https://drive.google.com/uc?export=view&id=1tkCAMUGMWEa-UauSHSG7MMV650Z0Ibil)


Сформируем бюджет суммы значений $f_d$: вычтем из общего кол-ва оборудования 7 единиц (сдвиг). Сдвиг делаем для дополнительной вариативности задачи. Полученные 38 единиц оборудования распределим пропорционально кол-ву тестов у отдела. Введем дополнительное ограничение снизу: каждому отделу должно достаться хотя бы 2 единицы оборудования. Значения $f_d$ приведены в таблице выше.

Значение весовых коэффициентов установим равными $w_d = 1$, т.е. нет приоритета между отделами. В условиях ограничения нагрузки и максимизации покрытия, приоритеты дают более тонкую настройку целевой, которой в нашем эксперименте можем пренебречь.

In [ ]:
# Загрузка и обработка входных данных
import pandas as pd
import numpy as np

df_sns = pd.read_csv("https://raw.githubusercontent.com/Lozkins/ORP/master/data/input/01_subnetworks.csv", sep=";", encoding="cp1251")
coverage_rate = 0.60  # Процент покрытия тестов отдела
device_cnt_shift = 7  # Сдвиг общего кол-ва оборудования для генерации нижней границы кол-ва оборудования для отдела
device_cnt = df_sns["device_id"].nunique()

# Генерируем значения для справедливого дележа (минимальное кол-во оборудования для отдела)
dct_tc_parts = (
    df_sns
    .pipe(lambda _df: _df.groupby(["dep_id"], as_index=False)["tc_id"].nunique())
    .assign(tc_part=lambda _df: _df["tc_id"] / _df["tc_id"].sum())
    .assign(device_min=lambda _df: round(_df["tc_part"] * (device_cnt - device_cnt_shift)).astype(int))
    .assign(device_min=lambda _df: np.where(_df["device_min"] < 2, 2, _df["device_min"]))
    .set_index("dep_id")
    .to_dict()["device_min"]
)

In [ ]:
dct_tc_parts

{1: 2, 2: 5, 3: 8, 4: 10, 5: 13}

## Python код

Синтаксис описания моделей для CP и MIP в ORtools отличается, но схема задания модели похожа. Ниже приведу код решения задачи для cp-sat солвера.

Параметр `is_agg` подробнее разберу в разделе Численный эксперимент. Для случая `is_agg=True`, постановка задачи полностью соответствует математическому описанию выше.

- Решение задачи с помощью cp-sat солвера в ORtools.



In [ ]:
# Установим библиотеку ortools
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protob

In [ ]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model


def cp_sat(time_limit, is_agg=True):
  """
    Решение задачи разбиения сети на подсети с помощью cp-sat solver
  """

  m = cp_model.CpModel()

  # Инициализация переменных
  # Переменные привязки оборудования к отделу
  df_x = df_sns[["dep_id", "device_id"]].drop_duplicates()
  df_x["var_id"] = np.ogrid[:df_x.shape[0]]
  df_x["var"] = df_x["var_id"].apply(lambda x: m.NewBoolVar(f"x_{x}"))

  # Переменная возможности выполнить тест
  df_y = df_sns[["dep_id", "tc_id"]].drop_duplicates()
  df_y["var_id"] = np.ogrid[:df_y.shape[0]]
  df_y["var"] = df_y["var_id"].apply(lambda y: m.NewBoolVar(f"y_{y}"))

  # Переменная возможности выполнить тест на определенной подсети
  df_z = df_sns[["dep_id", "tc_id", "sn_id"]].drop_duplicates()
  df_z["var_id"] = np.ogrid[:df_z.shape[0]]
  df_z["var"] = df_z["var_id"].apply(lambda s: m.NewBoolVar(f"s_{s}"))

  # Слаковая переменная - объем нарушения уровня покрытия тестов для отдела
  df_p = df_sns.groupby("dep_id", as_index=False)["tc_id"].nunique()
  df_p = df_p.rename({"tc_id": "w"}, axis=1)  # Добавляем веса для целевой
  df_p["var_id"] = np.ogrid[:df_p.shape[0]]
  df_p["var"] = df_p.apply(lambda p: m.NewIntVar(0, round(p.w * coverage_rate), f"p_{p.var_id}"), axis=1)

  # Добавляем ограничения в модель
  # Ограничение: Тестовый сценарий отдела можно выполнить на подсети, если все оборудование подсети закреплено за отделом
  dct_x = df_x.set_index(["dep_id", "device_id"]).to_dict()["var"]
  # Добавим оборудование в таблицу
  df_tmp = df_z.merge(df_sns[["dep_id", "tc_id", "sn_id", "device_id"]], how="left",
                      on=["dep_id", "tc_id", "sn_id"])
  df_cnstr_1 = df_tmp.groupby(["dep_id", "device_id"], as_index=False).agg({"var": "sum", "sn_id": "count"})
  if is_agg:
    for cnstr in df_cnstr_1.itertuples():
      key = cnstr.dep_id, cnstr.device_id
      m.Add(cnstr.var <= cnstr.sn_id * dct_x.get(key, 0))
  else:
    for cnstr in df_tmp.itertuples():
      key = cnstr.dep_id, cnstr.device_id
      m.Add(cnstr.var <= dct_x.get(key, 0))

  # Ограничение: Каждая единица оборудования может быть закреплена не более чем за одним отделом
  df_cnstr_2 = df_x.groupby("device_id").agg({"var": "sum"})
  for cnstr in df_cnstr_2.itertuples():
    m.Add(cnstr.var <= 1)

  # Ограничение: Тестовый сценарий отдела можно выполнить, если хотя бы одна подсеть использует оборудование полностью закрепленное за этим отделом
  dct_y = df_y.set_index(["dep_id", "tc_id"]).to_dict()["var"]
  df_cnstr_3 = df_z.groupby(["dep_id", "tc_id"], as_index=False).agg({"var": "sum"})
  for cnstr in df_cnstr_3.itertuples():
    key = cnstr.dep_id, cnstr.tc_id
    m.Add(cnstr.var >= dct_y.get(key, 0))

  # Ограничение: Условие покрытия объема тестов для каждого отдела
  dct_p = df_p.set_index("dep_id").to_dict()["var"]
  df_cnstr_4 = df_y.groupby("dep_id", as_index=False).agg({"var": sum, "tc_id": "count"})
  for cnstr in df_cnstr_4.itertuples():
    m.Add(cnstr.var + dct_p.get(cnstr.dep_id, 0) >= round(cnstr.tc_id * coverage_rate))

  # Ограничение: Условие справедливого дележа согласно объему тестов
  df_constr_5 = df_x.groupby("dep_id", as_index=False).agg({"var": "sum"})
  for cnstr in df_constr_5.itertuples():
    m.Add(cnstr.var >= dct_tc_parts.get(cnstr.dep_id, 0))

  # Целевая функция: минимизация взвешенного объема нарушений покрытия тестовых сценариев
  m.Minimize(sum(df_p["var"]))

  # Инициализация solver
  solver = cp_model.CpSolver()

  # Устанавливаем ограничения солвера
  solver.parameters.max_time_in_seconds = time_limit
  solver.parameters.log_search_progress = True

  # Решение задачи
  status = solver.Solve(m)

  # Проверяем статус
  print("Найдено оптимальное решение: ", status == cp_model.OPTIMAL)
  print('Целевая функция = %f' % solver.ObjectiveValue())

  # Извлекаем решение
  df_x["sol"] = df_x["var"].apply(lambda x: solver.value(x))
  df_y["sol"] = df_y["var"].apply(lambda x: solver.value(x))
  df_z["sol"] = df_z["var"].apply(lambda x: solver.value(x))
  df_p["sol"] = df_p["var"].apply(lambda x: solver.value(x))

  return dict(df_x=df_x, df_y=df_y, df_z=df_z, df_p=df_p)

# Численный эксперимент

Производительность open source солверов ниже чем у коммерческих (справедливо для солверов из одной парадигмы математического программирования). Задача по объему приближена к реальной, поэтому устанавливаю ограничение по времени на длительность поиска оптимального решения равное 1 часу. Рассмотрим два вида реализации задачи:

- Реализация в парадигме CP-SAT;
- Реализация в парадигме MIP.

Набор солверов следующий: [SCIP](https://www.scipopt.org/), коммерческий солвер (MIP) и [cp-sat](https://developers.google.com/optimization/cp/cp_solver?hl=en) (CP). Два солвера open source и один коммерческий.

Размерность исходной задачи: 985 ограничений и 19702 бинарные переменные и 5 целочисленных переменных (случай `is_agg=True`).

## Альтернативная модель

Построение модели - процесс творческий. Математическая формулировка задачи влияет на скорость поиска решения. В рамках эксперимента продемонстрирую влияние этого фактора на производительность солверов.

Рассмотрим ограничения типа (1): Тестовый сценарий отдела можно выполнить на подсети, если все оборудование подсети закреплено за отделом.

$$\sum_{i, j \in \Omega^{kd}}  z_{ijd} \le |\Omega^{kd}| x_{kd}, \quad \forall k \in K, d \in D.$$  

Левая часть представляет собой сумму допустимых подсетей отдела. Если хотя бы одно оборудование не находится в юрисдикции отдела, то эта сумма должна равняться 0. Предлагаю перейти к следующему набору ограничений:

$$z_{ijd} \le  x_{kd}, \quad \forall k \in K, d \in D, (i, j) \in \Omega^{kd}.$$

Логика ограничения не изменилась, но кол-во ограничений увеличится, т.к. для каждой допустимой подсети будет создан набор ограничений равный кол-ву единиц оборудования в топологии тестового сценария.

При замене ограничений (1) на новые получим задачу следующей размерности: 101258 ограничений и 19702 бинарных переменных и 5 целочисленных переменных (случай `is_agg=False`). Кол-во ограничений в модели выросло в ~106 раз. Целесообразность такого перехода рассудит эксперимент.

## Производительность моделей и солверов

Рассмотрел три солвера: cp, scip и com; каждый решал две задачи: agg (исходное ограничение (1)) и unzip (распакованное ограничение (1)). На графике выведены значения целевой функции (по оси y) из логов солверов с течением времени (ось x). Все солверы запускал с базовыми настройками. Значение целевой функции - размер нарушения покрытия, выраженное в кол-ве тестовых сценариев, которое требуется для удовлетворения требуемого покрытия.

![](https://drive.google.com/uc?export=view&id=1AjndFM_O_6-gLWP84MYZuqAB4YJyf7ZO)


Разберем аутсайдера эксперимента: scip. Первое решение находит спустя 1959 сек. и 3261 сек. для agg и unzip моделей, соответственно. Низкую производительность солвера можно связать с однопоточностью солвера на этапе решения MIP задачи.

В логах солвера SCIP при решении задачи agg можно обнаружить следующую запись:

` 10 deleted vars, 235 deleted constraints, 100498 added constraints, 5 tightened bounds, 0 added holes, 0 changed sides, 4 changed coefficients`

Солвер scip добавил в модель 100498 ограничений на этапе presolve. Это наводит на мысль, что солвер сам распаковал ограничения типа (1). Если знакомы с внутренним устройством солвера scip, прошу прокомментировать догадку.

Коммерческий солвер сразу нашел допустимое решение задачи agg, для задачи unzip спустя 34 сек. Но решение в 265 непокрытых тестовых сценариев нашел быстрее в случае unzip (3483 сек.) против 3600 сек. для agg модели. Сходимость задачи agg лучше в первой половине в то время, как unzip сходит немного быстрее во второй половине отведенного времени.

Программирование в ограничениях оказалось наиболее подходящим для решения задачи даже с учетом использования open source солвера cp-sat. Помимо лучшего значения целевой функции (262 шт.) среди рассмотренных солверов, в случае unzip решение найдено через 479 сек. Поздравляем с победой в локальном benchmark'е!

Кол-во произведенных симуляций не велико, заключение не то чтобы объективно. Однако, формулировка с распакованным ограничением типа (1) - unzip, показывает лучшую сходимость и является более "удобной" для солверов (для данной задачи).

Провел две симуляции эксперимента (2х6=12шт. расчетов), поведение солверов имеет сходство. Ниже аналогичный график для второй симуляции.

![](https://drive.google.com/uc?export=view&id=19g3RHkkPFyKZ8-vpsk0j6dcVhqPb99fL)

Коммерческий и scip солверы работают достаточно стабильно, в то время как cp-sat переобулся и показал обратную динамику. Во втором прогоне лучшее решение 264 непокрытых сценариев на версии agg. Нестабильность - цена доступности.

Дополнительно, хочу отметить следующее: все солверы не смогли улучшить нижнюю границу решения и отклониться от уровня 0.






## Результат

Разберем решения первой симуляции. Доказанного оптимального решения получить не удалось в установленные лимиты времени. Это не было камнем преткновения для использования решения в промышленной эксплуатации, заказчик доволен и пользуется моделью.

Эксперименты с ограничением по времени в 12 часов тоже не привели к доказанному оптимальному решению. Но и улучшить значение целевой функции тоже не удалось. Выглядит, что задача доставляет трудности алгоритмам оценки нижней границы решения.

В таблице представлено кол-во не покрытых тестовых сценариев в разрезе разделов. Столбец **cover_req** требуемое кол-во тестов для покрытия. Больше всего пострадал первый отдел, у него ни один тест не покрыт. Лучшее покрытие у самого большого отдела (**dep_id** = 5). Настройка весов $w_d$ позволяет сместить приоритет с самого "крупного" отдела, кроме того более умная настройка $f_d$ так же позволит повысить вариабельность задачи (сделать дележ более демократичным).

|dep\_id|cover\_req|cp\_unzip|com\_unzip|com\_agg|cp\_agg|scip\_agg|scip\_unzip|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|1|26|26|26|26|26|26|26|
|2|53|51|50|49|51|50|53|
|3|88|73|70|73|74|83|88|
|4|110|82|91|86|86|101|100|
|5|145|30|28|31|31|63|63|
|**Total**|**422**|**262**|**265**|**265**|**268**|**323**|**330**|

Распределение кол-ва единиц оборудования между отделами одинаково во всех решениях `{1: 2, 2: 5, 3: 8, 4: 10, 5: 20}` (ограничение на минимальный объем привязки $f_d$: `{1: 2, 2: 5, 3: 8, 4: 10, 5: 13}`). Т.е. все свободное оборудование (сдвиг в 7 единиц) назначено самому "крупному" отделу.

![](https://drive.google.com/uc?export=view&id=1vP770tUf1HYWhQaV_guMjfXm8aifqbK4)



# Заключение

Разработали решение разбиения тестового стенда на непересекающиеся подсети. Инструмент условно не ограничен кол-вом разбиений и позволяет получить приемлемое решение в пределах часа. Имеет пару управляемых параметров $w_d$ и $f_d$ для настройки модели. Предоставляет возможность моделировать различные гипотезы с меньшими затратами в сравнении с ручным моделированием.

Время затраченное на разработку прототипа ~60 чел./ч. Разработку провели в "свободное" время от основной задачи. Инвестиции заказчика полностью оправданы.

В статье отразил базовую постановку задачи и ее решения. В пользование передали решение с дополнительными эвристиками по ускорение поиска решения.

Что пробовали:
- У cp солверов есть условные ограничения типа EnforceOnly, которые можно применить для модели из статьи. В моих экспериментах значимого ускорения от их использования не произошло.
- Вместо мягких ограничений на покрытие можно использовать жесткие. В этом случае, теряем надежность работы модели, т.к. можем получить infeasible. В практическом использовании потребуется логировать места нарушений и добавляем работы пользователю. Кроме этого, с жесткими ограничениями поиск решения затрудняется и возникает необходимость эвристически быстро находить допустимое решение.
- Задачу можно решать с помощью [метода генерации столбцов](https://habr.com/ru/articles/800527/). Кол-во допустимых разбиений стенда (столбцов) достаточно большое, поэтому здесь требуется дополнительный алгоритм умного построения столбцов для уменьшения вариантов (выстраивать ансамбль моделей).

Пути по ускорению поиска решения. Можно заниматься настройкой солверов под задачу, которые позволят ускорить поиск решений (например, в [статье](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4047072) описан auto-tuner для MIP, или [здесь](https://link.springer.com/article/10.1007/s43069-024-00327-7)). Альтернатива погружению в детали работы солвера - варьирование или декомпозиция модели на базе экспертных предположений. Мы пошли по второму пути и преуспели в этом.


# Ссылки

- Jupyter блокнот статьи
- [Задача о разбиении множества](https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html)

# Дополнение

In [ ]:
# Построение графика производительности солверов
import plotly.express as px
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Lozkins/ORP/master/data/output/01_all_solvers.csv", sep=",", encoding="cp1251")

dct_cols = {"time": "Время, сек.", "obj": "Целевая функция, шт."}
df["name"] = df["type"] + "_" + df["cnstr"]
df = df.rename(columns=dct_cols)
fig = px.line(df, x='Время, сек.', y='Целевая функция, шт.', color='name', markers=True)
fig.show()

In [ ]:
# Построение картинки сети со случайными соединениями
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from random import  uniform, seed

df_sns = pd.read_csv("https://raw.githubusercontent.com/Lozkins/ORP/master/data/input/01_subnetworks.csv", sep=";", encoding="cp1251")
df_x = pd.read_csv("https://raw.githubusercontent.com/Lozkins/ORP/master/data/output/01_cp_unzip_df_x.csv", sep=",", encoding="cp1251")

df_tmp = df_x[df_x["sol"] == 1]
dct_dd = df_tmp.set_index("device_id").to_dict()["dep_id"]

seed(27)

df_tmp = df_x[df_x["sol"] == 1]
dct_dd = df_tmp.set_index("device_id").to_dict()["dep_id"]
dct_bounds = {1: [[0.4, 0.6], [0.4, 0.6]],
              2: [[0.0, 0.4], [0.0, 0.4]],
              3: [[0.6, 1.0], [0.6, 1.0]],
              4: [[0.6, 1.0], [0.0, 0.4]],
              5: [[0.0, 0.4], [0.6, 1.0]],}


uniform(-.5, .5)
devices = df_sns["device_id"].unique()
G = nx.random_geometric_graph(devices, 0.4, pos={i: (uniform(*dct_bounds[dct_dd[i]][0]),
                                                     uniform(*dct_bounds[dct_dd[i]][1])) for i in devices})

edge_x_same = []
edge_y_same = []
edge_x_diff = []
edge_y_diff = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    if dct_dd[edge[0]] == dct_dd[edge[1]]:
      edge_x_same.append(x0)
      edge_x_same.append(x1)
      edge_x_same.append(None)
      edge_y_same.append(y0)
      edge_y_same.append(y1)
      edge_y_same.append(None)
    else:
      edge_x_diff.append(x0)
      edge_x_diff.append(x1)
      edge_x_diff.append(None)
      edge_y_diff.append(y0)
      edge_y_diff.append(y1)
      edge_y_diff.append(None)


edge_trace_same = go.Scatter(
    x=edge_x_same, y=edge_y_same,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines', name="Internal Edges")

edge_trace_diff = go.Scatter(
    x=edge_x_diff, y=edge_y_diff,
    line=dict(width=0.5, color='#888', dash='dash'),  # , dash='dash'
    hoverinfo='none',
    mode='lines', name="External Edges")

node_x = []
node_y = []
colors = []
text = []
# dct_colors = {1: "green", 2: "green", 3: "green", 4: "green", 5: "green"}
dct_colors = {1: "white", 2: "black", 3: "blue", 4: "green", 5: "red"}
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
    colors.append(dct_colors[dct_dd[node]])
    text.append(node)


node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=colors,
        size=14,
        line_width=2), text=text, name="Nodes")


fig = go.Figure(data=[edge_trace_same, edge_trace_diff, node_trace],
             layout=go.Layout(
                titlefont_size=16,
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
# Генерация входных данных
import pandas as pd
import numpy as np
from numpy.random import default_rng

lst_tcs_cnt = [44, 89, 147, 184, 248] # Кол-во тестов у каждого отдела
seed = 7
np.random.seed(seed)

dct_tcs = {}

i = 1
lst_info = []
for tcs_cnt in lst_tcs_cnt:
  for tc_id in range(tcs_cnt):
    device_cnt = np.random.randint(1, 7)  # Кол-во оборудования требуемое для теста
    tester_cnt = np.random.randint(1, 3)  # Кол-во тестеров требуемое для теста
    sg_cnt = np.random.randint(10, 50)  # Кол-во допустимых назначений (подсетей) теста на стенде
    lst_info.append([i, tc_id, device_cnt, tester_cnt, sg_cnt])
  i += 1

df_info = pd.DataFrame(lst_info, columns=["dep_id", "tc_id", "device_cnt", "tester_cnt", "sg_cnt"])

rng = default_rng()
lst_sns = []

for row in df_info.itertuples():
  for sg_id in range(row.sg_cnt):
    devices_in = rng.choice(32, size=row.device_cnt, replace=False)  # Идентификатор оборудования
    tester_in = rng.choice(13, size=row.tester_cnt, replace=False)  # Идентификатор теста
    for device in devices_in:
      lst_sns.append([row.dep_id, row.tc_id, sg_id, f"d_{device}"])
    for tester in tester_in:
      lst_sns.append([row.dep_id, row.tc_id, sg_id, f"t_{tester}"])

df_sns = pd.DataFrame(lst_sns, columns=["dep_id", "tc_id", "sg_id", "device_id"])

df_sns_set = df_sns.groupby(["dep_id", "tc_id", "sg_id"], as_index=False).agg({"device_id": set})
df_tmp = df_sns_set.drop_duplicates("device_id")

df_sns = df_sns.merge(df_tmp[["dep_id", "tc_id", "sg_id"]], how="inner",
                      on=["dep_id", "tc_id", "sg_id"])
print(f"before {df_sns_set.shape[0]}")
print(f"after {df_tmp.shape[0]}")

before 21044
after 18763
